# Create Compute Engines that run SEIR

## Step 1: Create Service Account

1. Go to **Google Cloud Console**.
2. Search for **IAM**, and select **IAM**. Navigate to **Service Accounts** on the left.
3. Create a new **Service Account**.
4. Navigate back to **IAM** in the left panel and add the following permissions to the service account:
   - **Compute Admin**
   - **Compute Instance Admin (v1)**
   - **Compute Network Admin**
   - **Service Account User**
   - **Storage Admin**
5. Navigate back to **Service Accounts** and save an **API key** for the service account.
6. Save the API key to your computer. It will be used in later steps.

---

## Step 2: GCP Products Setup

For a new project:
1. **Enable Compute Engine**.
2. **Enable Storage**.
3. **Enable Artifact Registry**.

---

## Step 3: Create Docker Image

1. Download the `seir_docker` folder to your computer.
2. Open a terminal and navigate inside the `seir_docker` folder.
3. Build the Docker image by running:
   ```bash
   docker build -t seir .
4. gcloud auth login
5. docker tag seir gcr.io/{project_name}/seir
6. docker push gcr.io/{project_name}/seir

In [ ]:
from google.cloud import compute_v1
from google.oauth2 import service_account

# Load credentials for the InstancesClient
credentials = service_account.Credentials.from_service_account_file(
    "C:/Users/00011/Downloads/quiet-fusion-440218-i2-6ca4b9d26c62.json"
)
instances_client = compute_v1.InstancesClient(credentials=credentials)

def create_instance_with_docker(
    project_id: str,
    zone: str,
    instance_name: str,
    machine_type: str,
    image_family: str,
    image_project: str,
    docker_image: str,
    beta: float,
    epsilon: float,
    simulations: int,
    days: int
) -> compute_v1.Instance:
    """
    Creates a Compute Engine VM instance with full API access that pulls and runs a Docker container.

    Args:
        project_id: ID or number of the project you want to use.
        zone: Name of the zone you want to check, for example: us-west3-b.
        instance_name: Name of the new instance.
        machine_type: Machine type for the VM, e.g., "e2-medium".
        image_family: Image family for the VM OS, e.g., "debian-12".
        image_project: Google Cloud project hosting the OS image.
        docker_image: Docker container image to pull and run.
        beta: Value for the beta environment variable.
        epsilon: Value for the epsilon environment variable.

    Returns:
        Instance object.
    """
    startup_script = f"""#!/bin/bash
    sudo apt-get update
    sudo apt-get install -y docker.io
    sudo gcloud auth configure-docker
    sudo docker pull {docker_image}
    sudo docker run -e BETA={beta} -e EPSILON={epsilon} -e SIMULATIONS={simulations} -e DAYS={days} {docker_image}
    """

    metadata_items = [
        compute_v1.Items(key="startup-script", value=startup_script)
    ]

    # Define the disk for the VM using Debian 12 Bookworm First One I Found that worked, Could be changed for different workloads
    initialize_params = compute_v1.AttachedDiskInitializeParams(
        source_image="projects/debian-cloud/global/images/debian-12-bookworm-v20241112",
        disk_size_gb=10,  # Specify 10GB disk size
        disk_type=f"zones/{zone}/diskTypes/pd-balanced" 
    )
    disk = compute_v1.AttachedDisk(
        boot=True,
        auto_delete=True,
        initialize_params=initialize_params
    )

    # Define the network interface with an external IP Important for Downloading 
    access_config = compute_v1.AccessConfig(name="External NAT", type_="ONE_TO_ONE_NAT")
    network_interface = compute_v1.NetworkInterface(
        name="global/networks/default",
        access_configs=[access_config] 
    )

    # Define the instance
    instance_resource = compute_v1.Instance(
        name=instance_name,
        machine_type=f"zones/{zone}/machineTypes/{machine_type}",
        disks=[disk],
        network_interfaces=[network_interface],
        metadata=compute_v1.Metadata(items=metadata_items),
        service_accounts=[
            compute_v1.ServiceAccount(
                email="default",
                scopes=["https://www.googleapis.com/auth/cloud-platform"] # Important for Pulling Docker Image
            )
        ]
    )

    # Insert the instance
    instance_insert_request = compute_v1.InsertInstanceRequest(
        project=project_id,
        zone=zone,
        instance_resource=instance_resource
    )
    operation = instances_client.insert(request=instance_insert_request)
    operation.result()  # Wait for the operation to complete

    return instances_client.get(project=project_id, zone=zone, instance=instance_name)


In [15]:
create_instance_with_docker(
    project_id="quiet-fusion-440218-i2",
    zone="us-central1-a",
    instance_name="my-docker-vm8",
    machine_type="e2-medium",
    image_family="debian-12",
    image_project="debian-cloud",
    docker_image="gcr.io/quiet-fusion-440218-i2/seir",
    beta=2,
    epsilon=0.5,
    simulations=30,
    days=100
)


id: 210154080051540139
kind: "compute#instance"
name: "my-docker-vm8"
tags {
  fingerprint: "42WmSpB8rSM="
}
zone: "https://www.googleapis.com/compute/v1/projects/quiet-fusion-440218-i2/zones/us-central1-a"
shielded_instance_config {
  enable_secure_boot: false
  enable_vtpm: true
  enable_integrity_monitoring: true
}
creation_timestamp: "2024-11-21T11:47:17.173-08:00"
network_interfaces {
  kind: "compute#networkInterface"
  name: "nic0"
  access_configs {
    kind: "compute#accessConfig"
    name: "External NAT"
    type_: "ONE_TO_ONE_NAT"
    nat_i_p: "34.134.36.117"
    network_tier: "PREMIUM"
  }
  network_i_p: "10.128.0.35"
  network: "https://www.googleapis.com/compute/v1/projects/quiet-fusion-440218-i2/global/networks/default"
  fingerprint: "EmCdPKEgSp4="
  subnetwork: "https://www.googleapis.com/compute/v1/projects/quiet-fusion-440218-i2/regions/us-central1/subnetworks/default"
  stack_type: "IPV4_ONLY"
}
metadata {
  kind: "compute#metadata"
  items {
    key: "startup-scrip